Vladimir Kantorovich
345027767

# **filter**

clearing data from errors

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data from the parquet file
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')
train.head()

,age,boat,body,cabin,embarked,fare,home.dest,name,parch,pclass,sex,sibsp,survived,ticket
199,-1,C,-1,Unknown,0,15.2458,Unknown,"Moubarek, Master. Halim Gonios ('William George')",1,2,0,1,1,2661
706,12,C,-1,Unknown,0,11.2417,Unknown,"Nicola-Yarred, Master. Elias",0,2,0,1,1,2651
555,18,Unknown,-1,Unknown,2,13.0000,"Kontiolahti, Finland / Detroit, MI","Hiltunen, Miss. Marta",1,1,1,1,0,250650
382,37,Unknown,-1,Unknown,2,9.5875,Unknown,"Laitinen, Miss. Kristina Sofia",0,2,1,0,0,4135
316,41,Unknown,-1,Unknown,2,39.6875,Unknown,"Panula, Mrs. Juha (Maria Emilia Ojala)",5,2,1,0,0,3101295


In [77]:
test.head()

,age,boat,body,cabin,embarked,fare,home.dest,name,parch,pclass,sex,sibsp,survived,ticket
670,64,Unknown,263,Unknown,2,26.0000,"Isle of Wight, England","Nicholson, Mr. Arthur Ernest",0,0,0,0,0,693
251,48,14,-1,Unknown,2,36.7500,"St Ives, Cornwall / Hancock, MI","Davies, Mrs. John Morgan (Elizabeth Agnes Mary...",2,1,1,0,1,C.A. 33112
1225,35,Unknown,-1,Unknown,1,12.3500,Unknown,"Keane, Mr. Daniel",0,1,0,0,0,233734
300,19,Unknown,-1,C23 C25 C27,2,263.0000,"Winnipeg, MB","Fortune, Mr. Charles Alexander",2,0,0,3,0,19950
352,4,15,-1,Unknown,2,11.1333,Unknown,"Johnson, Master. Harold Theodor",1,2,0,1,1,347742


In [78]:
unique_values = train['body'].unique()
print(unique_values)

[ -1 201 175 314 143  67 287  61  43  98 328  16 298  96 292 101  97   9
 166 209  14  38   4 133 174 207 260  81 305 171  18 173  15 295 103  46
  19 261 124 284 131 156 232 206 110 294 135  50 258  17 148 293 190 322
  89 130  69 197  70  32 269  68 142 297 271  79 196 255 306 312  52 109
 165  75 304 169 249 259 120 119 149 181  35 236 188 172 299   1  80 275
 147 245 126  22  72]


In [79]:

train = train.reset_index()
train = train.rename(columns={'index': 'PassengerId'})
train = train.set_index('PassengerId')
print(train.head())


             age     boat  body    cabin  embarked     fare  \
PassengerId                                                   
199           -1        C    -1  Unknown         0  15.2458   
706           12        C    -1  Unknown         0  11.2417   
555           18  Unknown    -1  Unknown         2  13.0000   
382           37  Unknown    -1  Unknown         2   9.5875   
316           41  Unknown    -1  Unknown         2  39.6875   

                                      home.dest  \
PassengerId                                       
199                                     Unknown   
706                                     Unknown   
555          Kontiolahti, Finland / Detroit, MI   
382                                     Unknown   
316                                     Unknown   

                                                          name  parch  pclass  \
PassengerId                                                                     
199          Moubarek, Master. Halim G

In [80]:
def replace_age_outside_range(df):
    df['age'] = np.where((df['age'] >= 0) & (df['age'] <= 120), df['age'], -1)
    return df

In [81]:
train = replace_age_outside_range(train)
test = replace_age_outside_range(test)

In [82]:
def clean_ticket_column(df):
    def process_ticket(value):
        # Check if value is a number
        if isinstance(value, (int, float)):
            return value
        # Remove non-numeric characters
        elif isinstance(value, str):
            numeric_value = ''.join(filter(str.isdigit, value))
            return int(numeric_value) if numeric_value else -1
        # Replace anything else with -1
        else:
            return -1

    df['ticket'] = df['ticket'].apply(process_ticket)
    return df

# Apply the function to both train and test datasets
train = clean_ticket_column(train)
test = clean_ticket_column(test)

In [83]:
def count_unique_values(df, column_name):
    unique_counts = df[column_name].value_counts().to_dict()
    return unique_counts


In [84]:

# Specify the column name to process
column_to_process = 'boat'

# Get unique value counts for both train and test datasets
train_unique_counts = count_unique_values(train, column_to_process)
test_unique_counts = count_unique_values(test, column_to_process)

# Display the unique value counts
print("Train dataset unique value counts for column", column_to_process)
print(train_unique_counts)

print("Test dataset unique value counts for column", column_to_process)
print(test_unique_counts)

Train dataset unique value counts for column boat
{'Unknown': 655, '15': 34, '13': 32, 'C': 29, '10': 27, '14': 26, '4': 25, '5': 23, '16': 20, '11': 19, '3': 19, '8': 19, 'D': 18, '7': 17, '12': 17, '6': 15, '9': 14, '2': 10, 'A': 8, 'B': 8, '1': 4, '13 15': 2, '5 7': 2, '5 9': 1, '13 15 B': 1, '8 10': 1, '15 16': 1}
Test dataset unique value counts for column boat
{'Unknown': 168, '9': 11, 'C': 9, '3': 7, '13': 7, '14': 7, '4': 6, '11': 6, '7': 6, '6': 5, '5': 4, '8': 4, '16': 3, '2': 3, 'A': 3, '15': 3, '10': 2, 'C D': 2, '12': 2, 'D': 2, '1': 1, 'B': 1}


In [85]:
def replace_values(df, column_name, replacement_dict):
    df[column_name] = df[column_name].replace(replacement_dict)
    return df


replacement_dict = {'Unknown':0,'A': 100, 'B': 101, 'C': 102, 'D': 103}

# Apply the replacement function to both train and test datasets
train = replace_values(train, column_to_process, replacement_dict)
test = replace_values(test, column_to_process, replacement_dict)

In [86]:
column_to_process = 'cabin'
train_unique_counts = count_unique_values(train, column_to_process)
test_unique_counts = count_unique_values(test, column_to_process)

# Display the unique value counts
print("Train dataset unique value counts for column", column_to_process)
print(train_unique_counts)

print("Test dataset unique value counts for column", column_to_process)
print(test_unique_counts)

Train dataset unique value counts for column cabin
{'Unknown': 820, 'F4': 4, 'C23 C25 C27': 4, 'F33': 4, 'G6': 4, 'B57 B59 B63 B66': 4, 'F2': 4, 'E101': 3, 'C78': 3, 'D': 3, 'C22 C26': 3, 'E34': 3, 'C106': 2, 'C89': 2, 'B22': 2, 'B71': 2, 'C55 C57': 2, 'C6': 2, 'F G63': 2, 'E8': 2, 'C93': 2, 'C83': 2, 'B28': 2, 'D21': 2, 'C54': 2, 'C126': 2, 'E25': 2, 'B58 B60': 2, 'D19': 2, 'B49': 2, 'C85': 2, 'E33': 2, 'B96 B98': 2, 'C46': 2, 'D30': 2, 'D35': 2, 'C80': 2, 'E31': 2, 'D37': 2, 'C92': 2, 'B20': 2, 'B5': 2, 'C32': 2, 'D26': 2, 'E50': 2, 'C101': 2, 'E44': 2, 'D15': 2, 'D17': 2, 'D20': 2, 'B77': 2, 'D33': 2, 'A34': 2, 'C68': 2, 'C123': 2, 'E46': 2, 'E24': 2, 'A20': 1, 'A19': 1, 'D45': 1, 'C111': 1, 'C28': 1, 'B26': 1, 'C116': 1, 'D11': 1, 'B11': 1, 'B50': 1, 'F E69': 1, 'B73': 1, 'B61': 1, 'F E46': 1, 'C99': 1, 'C2': 1, 'A6': 1, 'C148': 1, 'E58': 1, 'A26': 1, 'B78': 1, 'D6': 1, 'C103': 1, 'C90': 1, 'B19': 1, 'B69': 1, 'C91': 1, 'E63': 1, 'B36': 1, 'C65': 1, 'E12': 1, 'F E57': 1, 'C7': 1, '

In [87]:
train[column_to_process] = train[column_to_process].apply(lambda x: 0 if x == "Unknown" else 1)
test[column_to_process] = test[column_to_process].apply(lambda x: 0 if x == "Unknown" else 1)

In [88]:
def clean_ticket_column(df):
    def process_ticket(value):
        # Check if value is a number
        if isinstance(value, (int, float)):
            return value
        # Remove non-numeric characters if it's a string
        elif isinstance(value, str):
            numeric_value = ''.join(filter(str.isdigit, value))
            return int(numeric_value) if numeric_value else -1
        # Replace anything else with -1
        else:
            return -1

    df['ticket'] = df['ticket'].apply(process_ticket)
    return df


In [89]:
column_to_process = 'fare'  # Replace 'your_column_name' with the actual column name

# Get unique value counts for both train and test datasets
train_unique_counts = count_unique_values(train, column_to_process)
test_unique_counts = count_unique_values(test, column_to_process)

# Display the unique value counts
print("Train dataset unique value counts for column", column_to_process)
print(train_unique_counts)

print("Test dataset unique value counts for column", column_to_process)
print(test_unique_counts)

Train dataset unique value counts for column fare
{8.050000190734863: 51, 13.0: 50, 7.75: 42, 26.0: 42, 7.8958001136779785: 38, 10.5: 28, 7.925000190734863: 21, 7.775000095367432: 20, 26.549999237060547: 20, 8.662500381469727: 19, 7.2291998863220215: 19, 7.224999904632568: 16, 7.25: 15, 7.8541998863220215: 14, 0.0: 13, 21.0: 12, 14.5: 10, 69.55000305175781: 9, 16.100000381469727: 9, 7.79580020904541: 9, 15.5: 9, 14.45419979095459: 9, 56.49580001831055: 8, 9.5: 8, 27.720800399780273: 8, 52.0: 8, 23.0: 7, 13.5: 7, 31.274999618530273: 7, 15.245800018310547: 7, 7.050000190734863: 7, 7.550000190734863: 6, 53.099998474121094: 6, 39.0: 6, 27.899999618530273: 6, 11.5: 6, 30.0: 6, 7.73330020904541: 6, 24.149999618530273: 6, 7.879199981689453: 6, 46.900001525878906: 6, 39.6875: 5, 262.375: 5, 27.75: 5, 31.387500762939453: 5, 21.075000762939453: 5, 151.5500030517578: 5, 83.1583023071289: 5, 52.55419921875: 4, 14.45829963684082: 4, 34.375: 4, 7.650000095367432: 4, 90.0: 4, 263.0: 4, 73.5: 4, 12.35

In [90]:
def replace_negative_non_numeric(df, column_name="column_to_process"):
    def process_value(value):
        # Check if value is numeric
        if pd.api.types.is_numeric_dtype(type(value)):
            # If numeric, check if it's negative
            return value if value >= 0 else -1
        else:
            # If not numeric, replace with -1
            return -1

    df[column_name] = df[column_name].apply(process_value)
    return df

In [91]:
train = replace_negative_non_numeric(train, column_to_process)
test = replace_negative_non_numeric(test, column_to_process)


In [92]:
column_to_process = 'home.dest'
train_unique_counts = count_unique_values(train, column_to_process)
test_unique_counts = count_unique_values(test, column_to_process)

# Display the unique value counts
print("Train dataset unique value counts for column", column_to_process)
print(train_unique_counts)

print("Test dataset unique value counts for column", column_to_process)
print(test_unique_counts)

Train dataset unique value counts for column home.dest
{'Unknown': 443, 'New York, NY': 54, 'London': 10, 'Cornwall / Akron, OH': 9, 'Sweden Winnipeg, MN': 7, 'Philadelphia, PA': 7, 'Brooklyn, NY': 7, 'Wiltshire, England Niagara Falls, NY': 6, 'Montreal, PQ': 6, 'Winnipeg, MB': 6, 'Paris, France': 5, 'Bournmouth, England': 4, 'Guntur, India / Benton Harbour, MI': 4, 'Rotherfield, Sussex, England Essex Co, MA': 4, 'Guernsey / Elizabeth, NJ': 4, 'Haverford, PA / Cooperstown, NY': 4, 'Ottawa, ON': 4, 'Sweden Worcester, MA': 4, 'Devon, England Wichita, KS': 4, 'Paris / Montreal, PQ': 4, 'Belfast': 4, 'Haverford, PA': 4, 'Ruotsinphytaa, Finland New York, NY': 4, 'Montevideo, Uruguay': 3, 'Lakewood, NJ': 3, 'Greenport, NY': 3, 'Southampton': 3, 'Hudson, NY': 3, 'Elkins Park, PA': 3, 'Ireland Chicago, IL': 3, 'Duluth, MN': 3, 'Hong Kong New York, NY': 3, 'East Providence, RI': 3, 'Minneapolis, MN': 3, 'Los Angeles, CA': 3, 'Syria Kent, ON': 3, 'Youngstown, OH': 3, 'Paris / Haiti': 3, 'Milwauk

In [93]:
train[column_to_process] = train[column_to_process].apply(lambda x: 0 if x == "Unknown" else 1)
test[column_to_process] = test[column_to_process].apply(lambda x: 0 if x == "Unknown" else 1)

In [94]:
column_to_process = "boat"
train_unique_counts = count_unique_values(train, column_to_process)
test_unique_counts = count_unique_values(test, column_to_process)

# Display the unique value counts
print("Train dataset unique value counts for column", column_to_process)
print(train_unique_counts)

print("Test dataset unique value counts for column", column_to_process)
print(test_unique_counts)

Train dataset unique value counts for column boat
{0: 655, '15': 34, '13': 32, 102: 29, '10': 27, '14': 26, '4': 25, '5': 23, '16': 20, '11': 19, '3': 19, '8': 19, 103: 18, '7': 17, '12': 17, '6': 15, '9': 14, '2': 10, 100: 8, 101: 8, '1': 4, '13 15': 2, '5 7': 2, '5 9': 1, '13 15 B': 1, '8 10': 1, '15 16': 1}
Test dataset unique value counts for column boat
{0: 168, '9': 11, 102: 9, '3': 7, '13': 7, '14': 7, '4': 6, '11': 6, '7': 6, '6': 5, '5': 4, '8': 4, '16': 3, '2': 3, 100: 3, '15': 3, '10': 2, 'C D': 2, '12': 2, 103: 2, '1': 1, 101: 1}


# Cross validation

In [95]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

We use the RandomForestClassifier model with 50 trees, maximum tree depth 5. Anything more causes overfitting and less underfitting

In [96]:

feature_columns = ["age", "cabin", "embarked", "fare", "home.dest", "parch", "pclass", "sex", "sibsp", "ticket"]
target_column = "survived"

# Divide data into features and target column
X_train = train[feature_columns]
y_train = train[target_column]

X_test = test[feature_columns]
y_test = test[target_column]

model = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=1)
cv_scores = cross_val_score(model, X_train, y_train, cv=5)  # 5-fold cross-validation

print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())

# learning model
model.fit(X_train, y_train)


y_pred = model.predict(X_test)

#  accuracy of model
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)

print(classification_report(y_test, y_pred))

Cross-validation scores: [0.79047619 0.81428571 0.77990431 0.79425837 0.80382775]
Mean cross-validation score: 0.7965504670767828
Test accuracy: 0.8320610687022901
              precision    recall  f1-score   support

           0       0.81      0.96      0.88       163
           1       0.90      0.63      0.74        99

    accuracy                           0.83       262
   macro avg       0.85      0.79      0.81       262
weighted avg       0.84      0.83      0.82       262



# Hyperparameter optimization

In [97]:


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Parameters for RandomizedSearchCV (reduced range of parameters)
param_dist = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Initialize the model
model = RandomForestClassifier(random_state=1)

# RandomizedSearchCV for hyperparameter tuning (reduced number of iterations and folds)
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=20, cv=3, scoring='accuracy', n_jobs=-1, random_state=1)
random_search.fit(X_train, y_train)

# Best parameters and evaluation
print("Best parameters found by RandomizedSearchCV:")
print(random_search.best_params_)

# Train the model with the best parameters
best_model = random_search.best_estimator_

# Predict on test data with the best model
y_pred_best = best_model.predict(X_test)

# Evaluate the accuracy of the model on test data
accuracy_best = accuracy_score(y_test, y_pred_best)
print("Accuracy on test data with the best model:", accuracy_best)

# Print the full classification report
print(classification_report(y_test, y_pred_best))




Best parameters found by RandomizedSearchCV:
{'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 10}
Accuracy on test data with the best model: 0.8244274809160306
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       163
           1       0.84      0.67      0.74        99

    accuracy                           0.82       262
   macro avg       0.83      0.79      0.80       262
weighted avg       0.83      0.82      0.82       262



#k-means

In [98]:





# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Use K-Means for 2 clusters
kmeans = KMeans(n_clusters=2, random_state=1)
clusters_train = kmeans.fit_predict(X_train_scaled)
clusters_test = kmeans.predict(X_test_scaled)

X_train['cluster'] = clusters_train
X_test['cluster'] = clusters_test

model = RandomForestClassifier(n_estimators=100, random_state=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test data:", accuracy)
print(classification_report(y_test, y_pred))

test['predicted_survived'] = y_pred
survivors = test[test['predicted_survived'] == 1]
print("List of surviving passengers:")
survivors.head()


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-98-453e917d953b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['cluster'] = clusters_train
<ipython-input-98-453e917d953b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['cluster'] = clusters_test


Accuracy on test data: 0.8053435114503816
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       163
           1       0.78      0.68      0.72        99

    accuracy                           0.81       262
   macro avg       0.80      0.78      0.79       262
weighted avg       0.80      0.81      0.80       262

List of surviving passengers:


,age,boat,body,cabin,embarked,fare,home.dest,name,parch,pclass,sex,sibsp,survived,ticket,predicted_survived
251,48,14,-1,0,2,36.750000,1,"Davies, Mrs. John Morgan (Elizabeth Agnes Mary...",2,1,1,0,1,33112,1
300,19,0,-1,1,2,263.000000,1,"Fortune, Mr. Charles Alexander",2,0,0,3,0,19950,1
1061,23,6,-1,1,0,113.275002,1,"Newell, Miss. Marjorie",0,0,1,1,1,35273,1
1087,3,0,-1,0,2,13.775000,0,"Peacock, Miss. Treasteall",1,2,1,1,0,3101315,1
1100,31,3,-1,1,0,134.500000,0,"Wilson, Miss. Helen Alice",0,0,1,0,1,16966,1
